In [2]:
import socket
import threading
import argparse
import time

max = 50
n = 100

listlock = threading.Lock()
listNoFull = threading.Condition(listlock)
listNoEmpty = threading.Condition(listlock)

frequencylock = threading.Lock()

inindex = 0
outindex = 0
listupdated = False

class Listnode:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.booktitle = None
        self.processed = False

class Linkedlist:
    def __init__(self):
        self.head = None

sharedLinkedList = LinkedList()

class Bookfrequency:
    def __init__(self):
        self.title = None
        self.frequency = 0

bookfrequencies = [Bookfrequency() for i in range(MAX)]

def countpattern(text, pattern):
    return text.count(pattern)

def handleClient(clientSocket, clientId):
    try:
        booktitle = None
        clientdata = b""  
        clientbooklines = []

        while True:
            data = clientSocket.recv(1024)
            if not data:
                break
            clientdata += data  

            while b"\n" in clientdata:
                line, clientdata = clientdata.split(b"\n", 1)
                lineStr = line.decode("utf-8").strip()

                if booktitle is None:
                    booktitle = lineStr  
                    print(f"Client {clientId} sent book title: {booktitle}")
                clientbooklines.append(lineStr)

                node = Listnode(lineStr)
                node.booktitle = booktitle

                with listlock:
                    global inindex
                    while (inindex + 1) % N == outindex:
                        listnofull.wait()

                    if sharedlinkedlist.head is None:
                        sharedlinkedlist.head = node
                    else:
                        current = sharedlinkedlist.head
                        while current.next is not None:
                            current = current.next
                        current.next = node

                    inIndex = (inIndex + 1) % N
                    listnoempty.notify()

        bookFilename = f"book_{clientId:02d}.txt"
        with open(bookFilename, "w", encoding="utf-8") as bookFile:
            for line in clientBookLines:
                bookFile.write(line + "\n")
        print(f"Book saved for client {clientId} to {bookFilename}")

    except Exception as e:
        print(f"Error handling client {clientId}: {e}")
    finally:
        clientSocket.close()

def analysisThread(pattern):
    lastProcessed = None
    while True:
        with listLock:
            global outIndex
            while inIndex == outIndex:
                listNoEmpty.wait()

            currentNode = sharedLinkedList.head if lastProcessed is None else lastProcessed

            while currentNode is not None:
                if not currentNode.processed:
                    matchCount = countPattern(currentNode.data, pattern)
                    if matchCount > 0:
                        with frequencyLock:
                            for book in bookFrequencies:
                                if book.title == currentNode.bookTitle:
                                    book.frequency += matchCount
                                    break
                            else:
                                for book in bookFrequencies:
                                    if not book.title:
                                        book.title = currentNode.bookTitle
                                        book.frequency += matchCount
                                        break

                            global listUpdated
                            listUpdated = True
                    currentNode.processed = True

                outIndex = (outIndex + 1) % N
                listNoFull.notify()

                lastProcessed = currentNode
                currentNode = currentNode.next

def printFrequencies(pattern):
    while True:
        time.sleep(5)
        with frequencyLock:
            if listUpdated:
                sortedBooks = sorted(bookFrequencies, key=lambda x: -x.frequency)
                for book in sortedBooks:
                    if book.frequency > 0:
                        print(f"Book: {book.title}, Pattern: {pattern}, Frequency: {book.frequency}")
                listUpdated = False

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("-l", "--port", type=int, required=True)
    parser.add_argument("-p", "--pattern", type=str, required=True)
    args = parser.parse_args()

    host = "localhost"
    port = args.port
    pattern = args.pattern

    serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serverSocket.bind((host, port))
    serverSocket.listen(5)

    print(f"Server is listening on {host}:{port}")

    for i in range(2):
        threading.Thread(target=analysisThread, args=(pattern,)).start()

    threading.Thread(target=printFrequencies, args=(pattern,)).start()

    clientId = 1
    try:
        while True:
            clientSocket, clientAddress = serverSocket.accept()
            print(f"Accepted connection from {clientAddress}")
            threading.Thread(target=handleClient, args=(clientSocket, clientId)).start()
            clientId += 1
    except KeyboardInterrupt:
        print("Server interrupted. Closing server...")
    finally:
        serverSocket.close()

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] -l PORT -p PATTERN
ipykernel_launcher.py: error: the following arguments are required: -l/--port, -p/--pattern


SystemExit: 2